smartphone components:
1. screen
2. camera
3. soc
4. ram & rom
5. mainboard
6. sensor
7. battery

assemble:
location of the assem

production: 1000000

In [39]:
using JuMP, Gurobi

m = Model(solver=GurobiSolver(OutputFlag=0))
#1000000 total

total = 1000000 #production

component =[:screen,:camera,:soc,:ramrom,:mainboard,:sensor,:battery]


#need data 
screen = [200,100,100,100]
camera = [100,100,100,100]
soc = [100,100,100]
ramrom = [100,100,100,100]
mainboard = [100,100,100,100,100]
sensor = [100,100,100,100,100,100]
battery = [100,100,100,100,100]

#number of suppliers for each component
supplier = Dict(zip(component, [4,4,3,4,5,6,5]))
# non-defective
nondefective = Dict(zip(component, [0.4,0.2,0.05,0.1,0.1,0.05,0.1]))

totaleach = Dict(zip(component, [0,0,0,0,0,0,0]))
for i = component
    totaleach[i] = total*(1+nondefective[i])
end


println(totaleach)
expectation = Dict(zip(component, [0,0,0,0,0,0,0]))
for i = component
    expectation[i] = totaleach[i]/supplier[i]
end

println(expectation)

@variable(m, screen_x[1:4] >= 0)
@variable(m, camera_x[1:4] >= 0)
@variable(m, soc_x[1:3] >= 0)
@variable(m, ramrom_x[1:4] >= 0)
@variable(m, mainboard_x[1:5] >= 0)
@variable(m, sensor_x[1:6] >= 0)
@variable(m, battery_x[1:5] >= 0)

lambda = Dict(zip(component,[0.005, 0.002,0.006,0.004,0.001,0.001,0.0005])) 

@constraint(m, sum(screen_x[i] for i = 1:4)>= totaleach[:screen])
@constraint(m, sum(camera_x[i] for i = 1:4)>= totaleach[:camera])
@constraint(m, sum(soc_x[i] for i = 1:3)>= totaleach[:soc])
@constraint(m, sum(ramrom_x[i] for i = 1:4)>= totaleach[:ramrom])
@constraint(m, sum(mainboard_x[i] for i = 1:5)>= totaleach[:mainboard])
@constraint(m, sum(sensor_x[i] for i = 1:6)>= totaleach[:sensor])
@constraint(m, sum(battery_x[i] for i = 1:5)>= totaleach[:battery])

#screen_x[i]*screen[i] + lambda[:screen]*(sum((screen[i]-expectation[:screen])^2) for i = 1:4

println(lambda[:screen])
@objective(m, Min, 
     sum(screen_x[i]*screen[i] + lambda[:screen]*(screen_x[i]-expectation[:screen])^2 for i = 1:4)
    +sum(camera_x[i]*camera[i] + lambda[:camera]*(camera_x[i]-expectation[:camera])^2 for i = 1:4)
    +sum(soc_x[i]*soc[i] + lambda[:soc]*(soc_x[i]-expectation[:soc])^2 for i = 1:3)
    +sum(ramrom_x[i]*ramrom[i] + lambda[:ramrom]*(ramrom_x[i]-expectation[:ramrom])^2 for i = 1:4)
    +sum(mainboard_x[i]*mainboard[i] + lambda[:mainboard]*(mainboard_x[i]-expectation[:mainboard])^2 for i = 1:5)
    +sum(sensor_x[i]*sensor[i] + lambda[:sensor]*(sensor_x[i]-expectation[:sensor])^2 for i = 1:6)
    +sum(battery_x[i]*battery[i] + lambda[:battery]*(battery_x[i]-expectation[:battery])^2 for i = 1:5)
 )
solve(m)

println(getvalue(screen_x))
println(getvalue(camera_x))
println(getvalue(soc_x))
println(getvalue(ramrom_x))
println(getvalue(mainboard_x))
println(getvalue(sensor_x))
println(getvalue(battery_x))
println(getobjectivevalue(m))



Dict(:camera=>1200000,:screen=>1400000,:battery=>1100000,:sensor=>1050000,:ramrom=>1100000,:mainboard=>1100000,:soc=>1050000)
Dict(:camera=>300000,:screen=>350000,:battery=>220000,:sensor=>175000,:ramrom=>275000,:mainboard=>220000,:soc=>350000)
0.005
Academic license - for non-commercial use only
[342500.0, 352500.0, 352500.0, 352500.0]
[300000.0, 300000.0, 300000.0, 300000.0]
[350000.0, 350000.0, 350000.0]
[275000.0, 275000.0, 275000.0, 275000.0]
[220000.0, 220000.0, 220000.0, 220000.0, 220000.0]
[175000.0, 175000.0, 175000.0, 175000.0, 175000.0, 175000.0]
[220000.0, 220000.0, 220000.0, 220000.0, 220000.0]
8.34624999999999e8


In [73]:
using JuMP, Gurobi, Clp

x_pos = Dict(zip(component,[-180,200,400,300,230,-280,-500]))
y_pos = Dict(zip(component,[300,-260,250,430,380,600,-200]))

p = Dict(zip(component, [0.000002,0.000001,0.000001,0.000001,0.0000015,0.000001,0.00001]))
allowance_x = 0
allowance_y = 0

l = 20
r = 100

price_in = 2
price_out = 3

lambda =5


5

In [74]:
m = Model(solver = GurobiSolver(OutputFlag = 0))
@variable(m, x)
@variable(m, y)
@constraint(m, y<=20)
@constraint(m, y>=-20)
@constraint(m, x<=20)
@constraint(m, x>=-20)
@objective(m, Min, sum(((x-x_pos[i])^2 + (y-y_pos[i])^2)*p[i]*totaleach[i] for i = component)
    +total*price_in )

solve(m)

println(getobjectivevalue(m))
println(getvalue(x))
println(getvalue(y))

Academic license - for non-commercial use only
6.789345176322418e6
-20.0
16.14609571788413


In [75]:
m2 = Model(solver = GurobiSolver(OutputFlag = 0))
@variable(m2, x2)
@variable(m2, y2)

@constraint(m2, x2>=-20)
@constraint(m2, y2>= 20)
@constraint(m2, (x2-allowance_x)^2 + (y2-allowance_y)^2  <= r^2)

@objective(m2, Min, sum(((x2-x_pos[i])^2 + (y2-y_pos[i])^2)*p[i]*totaleach[i] for i = component) +
lambda*((allowance_x-x2)^2+(allowance_y-y2)^2) + price_out*total)

solve(m2)

println(getobjectivevalue(m2))
println(getvalue(x2))
println(getvalue(y2))

Academic license - for non-commercial use only
7.793640000186198e6
-19.999999980486457
19.999999981202834


In [76]:
m3 = Model(solver = GurobiSolver(OutputFlag = 0))
@variable(m3, x3)
@variable(m3, y3)

@constraint(m3, x3>=20)
@constraint(m3, y3<= 20)
@constraint(m3, (x3-allowance_x)^2 + (y3-allowance_y)^2  <= r^2)

@objective(m3, Min, sum(((x3-x_pos[i])^2 + (y3-y_pos[i])^2)*p[i]*totaleach[i] for i = component) +
lambda*((allowance_x-x3)^2+(allowance_y-y3)^2) + price_out*total)

solve(m3)

println(getobjectivevalue(m3))
println(getvalue(x3))
println(getvalue(y3))

Academic license - for non-commercial use only
8.186666388040743e6
19.999999971438893
12.89739728548153


In [77]:
m4 = Model(solver = GurobiSolver(OutputFlag = 0))
@variable(m4, x4)
@variable(m4, y4)

@constraint(m4, x4<=20)
@constraint(m4, y4<= -20)
@constraint(m4, (x4-allowance_x)^2 + (y4-allowance_y)^2  <= r^2)

@objective(m4, Min, sum(((x4-x_pos[i])^2 + (y4-y_pos[i])^2)*p[i]*totaleach[i] for i = component) +
lambda*((allowance_x-x4)^2+(allowance_y-y4)^2) + price_out*total)

solve(m4)

println(getobjectivevalue(m4))
println(getvalue(x4))
println(getvalue(y4))

Academic license - for non-commercial use only
7.2792551804144345e6
-97.97959023703322
-20.000000107634865


In [78]:
m5 = Model(solver = GurobiSolver(OutputFlag = 0))
@variable(m5, x5)
@variable(m5, y5)

@constraint(m5, x5<=-20)
@constraint(m5, y5>= -20)
@constraint(m5, (x5-allowance_x)^2 + (y5-allowance_y)^2  <= r^2)

@objective(m5, Min, sum(((x5-x_pos[i])^2 + (y5-y_pos[i])^2)*p[i]*totaleach[i] for i = component) +
lambda*((allowance_x-x5)^2+(allowance_y-y5)^2) + price_out*total)

solve(m5)

println(getobjectivevalue(m5))
println(getvalue(x5))
println(getvalue(y5))

Academic license - for non-commercial use only
7.244438013871507e6
-99.79014973483045
6.47499377364202
